# OPERA Direct S3 Access

Access OPERA L2-RTC and L2-CSLC in-place

Author: Alex Lewandowski; Alaska Satellite Facility

Notebook based on following ASF documentation and examples:
- [ASF OPERA Data Discovery](https://asf.alaska.edu/datasets/daac/opera/)
- [ASF S3 Credential Instructions](https://cumulus.asf.alaska.edu/s3credentialsREADME?)


In [1]:
import ast
from datetime import datetime
from getpass import getpass
import io
import json
import re
import urllib.request

import ipywidgets as widgets
from IPython.display import display

import boto3
import h5py
import pandas as pd
import s3fs
import xarray as xr

---

## 1. Select some OPERA product IDs

**L2-RTC-S1**

In [2]:
# code assumes opera_ids contains a single product type
opera_ids = [
    "OPERA_L2_RTC-S1_T137-292339-IW3_20240201T020001Z_20240201T081115Z_S1A_30_v1.0",
    "OPERA_L2_RTC-S1_T137-292339-IW3_20240201T020001Z_20240201T114538Z_S1A_30_v1.0",
    "OPERA_L2_RTC-S1_T137-292339-IW3_20240120T020002Z_20240120T142521Z_S1A_30_v1.0",
]

prefix = "OPERA_L2_RTC-S1" if "RTC" in opera_ids[0] else "OPERA_L2_CSLC-S1"

**L2-RTC-S1-STATIC**

In [3]:
# # code assumes opera_ids contains a single product type
# opera_ids = [
#     "OPERA_L2_RTC-S1-STATIC_T137-292339-IW3_20140403_S1A_30_v1.0",
# ]

# prefix = "OPERA_L2_RTC-S1" if "RTC" in opera_ids[0] else "OPERA_L2_CSLC-S1"

**L2-CSLC-S1**

In [4]:
# # code assumes opera_ids contains a single product type
# opera_ids = [
#     "OPERA_L2_CSLC-S1_T071-151218-IW3_20231023T135235Z_20231024T144245Z_S1A_VV_v1.0",
#     "OPERA_L2_CSLC-S1_T137-292339-IW3_20240201T020001Z_20240202T175818Z_S1A_VV_v1.0",
#     "OPERA_L2_CSLC-S1_T137-292339-IW3_20240201T020001Z_20240202T175821Z_S1A_VV_v1.0",
#     "OPERA_L2_CSLC-S1_T137-292339-IW3_20240120T020002Z_20240121T075842Z_S1A_VV_v1.0",
# ]

# prefix = "OPERA_L2_RTC-S1" if "RTC" in opera_ids[0] else "OPERA_L2_CSLC-S1"

**L2-CSLC-S1-STATIC**

In [5]:
# # code assumes opera_ids contains a single product type
# opera_ids = [
#     "OPERA_L2_CSLC-S1-STATIC_T137-292339-IW3_20140403_S1A_v1.0",
# ]

# prefix = "OPERA_L2_RTC-S1" if "RTC" in opera_ids[0] else "OPERA_L2_CSLC-S1"

---

## 2. Request S3 access credentials

**Enter your Earthdata Login Bearer Token**

[Instructions for creating an EDL bearer token](https://urs.earthdata.nasa.gov/documentation/for_users/user_token)

In [6]:
token = getpass("Enter your EDL Bearer Token")

Enter your EDL Bearer Token ········


In [7]:
event = {
    "CredentialsEndpoint": "https://cumulus.asf.alaska.edu/s3credentials",
    "BearerToken": token,
    "Bucket": "asf-cumulus-prod-opera-products",
    "Prefix": prefix,
    "StaticPrefix": f"{prefix}_STATIC"
}

In [8]:
# Get temporary download credentials
tea_url = event["CredentialsEndpoint"]
bearer_token = event["BearerToken"]
req = urllib.request.Request(
    url=tea_url,
    headers={"Authorization": f"Bearer {bearer_token}"}
)
with urllib.request.urlopen(req) as f:
    creds = json.loads(f.read().decode())

## 3. If accessing RTC or RTC-STATIC data, select a layer type

This is not necessary for CSLC and CSLC-STATIC, as their data layers are stored in multidimensional HDF5 files

In [9]:
rtc = 'RTC' in event["Prefix"]
static = 'STATIC' in opera_ids[0]

if rtc:
    if static:
        file = {
            'Incidence angle (ellipsoidal)': '_incidence_angle.tif',
            'Local-incidence angle': '_local_incidence_angle.tif', 
            'No. of looks': '_number_of_looks.tif',
            'Layover Shadow Mask layer': '_mask.tif',
            'RTC Area Normalization Factor (ANF) gamma0 to beta0': '_rtc_anf_gamma0_to_beta0.tif',
            'RTC Area Normalization Factor (ANF) gamma0 to sigma0': '_rtc_anf_gamma0_to_sigma0.tif'
        }
    else:
        file = {
            'VH RTC': '_VH.tif',
            'VV RTC': '_VV.tif',
            'Layover Shadow Mask layer': '_mask.tif',
        }
elif 'CSLC'  not in event['Prefix']:
    raise Exception("Unrecognized Product Type")

In [10]:
if rtc:
    print("Select a product type")
    product_choice = widgets.RadioButtons(
        options=file,
        description='',
        disabled=False,
        layout={'width': '500px'}
    )
    display(product_choice)

Select a product type


RadioButtons(layout=Layout(width='500px'), options={'VH RTC': '_VH.tif', 'VV RTC': '_VV.tif', 'Layover Shadow …

## 4. Open a single OPERA product

Access the first product in `opera_ids`

In [11]:
filename = f"{opera_ids[0]}{product_choice.value}" if rtc else f"{opera_ids[0]}.h5"
object_key = f"{event['StaticPrefix']}/{opera_ids[0]}/{filename}" if static else f"{event['Prefix']}/{opera_ids[0]}/{filename}" 

fs = s3fs.S3FileSystem(key=creds['accessKeyId'], secret=creds['secretAccessKey'], token=creds['sessionToken'])

# Define S3 path
s3_path = f"{event['Bucket']}/{object_key}"

# Open the file as a file-like object using s3fs
with fs.open(s3_path, mode='rb') as f:
    if '.h5' in object_key and not rtc:
        ds = xr.open_dataset(f, engine='h5netcdf', group="data") # can't seem to specify a specific group here, like "data/VV"
        with h5py.File(f, 'r') as h5f:
            attributes = h5f.attrs
            for attr, value in attributes.items():
                try:
                    ds.attrs[attr] = value.decode('utf-8')
                except AttributeError:
                    ds.attrs[attr] = value
    else:
        ds = xr.open_dataarray(f, engine="rasterio")
ds

<xarray.DataArray 'band_data' (band: 1, y: 1609, x: 3106)>
[4997554 values with dtype=float32]
Coordinates:
  * band         (band) int64 1
  * x            (x) float64 7.713e+05 7.713e+05 ... 8.644e+05 8.644e+05
  * y            (y) float64 4.228e+06 4.228e+06 4.228e+06 ... 4.18e+06 4.18e+06
    spatial_ref  int64 ...
Attributes: (12/98)
    ABSOLUTE_ORBIT_NUMBER:                                                   ...
    ACQUISITION_MODE:                                                        ...
    AREA_OR_POINT:                                                           ...
    BOUNDING_BOX:                                                            ...
    BOUNDING_BOX_EPSG_CODE:                                                  ...
    BOUNDING_BOX_PIXEL_COORDINATE_CONVENTION:                                ...
    ...                                                                               ...
    STATIC_LAYERS_DATA_ACCESS:                                               ...
    SUB_SWATH_ID:                                                            ...
    TRACK_NUMBER:                                                            ...
    ZERO_DOPPLER_END_TIME:                                                   ...
    ZERO_DOPPLER_START_TIME:                                                 ...
    long_name:                                                               ...

---

## 5. Create an RTC time-series 

**Build a sorted list of OPERA L2-RTC-S1 IDs**

In [12]:
opera_rtc_ids = sorted([
    'OPERA_L2_RTC-S1_T137-292339-IW3_20231004T020005Z_20240122T203351Z_S1A_30_v1.0',
    'OPERA_L2_RTC-S1_T137-292339-IW3_20231016T020005Z_20231016T154509Z_S1A_30_v1.0',
    'OPERA_L2_RTC-S1_T137-292339-IW3_20231028T020005Z_20231029T045555Z_S1A_30_v1.0',
    'OPERA_L2_RTC-S1_T137-292339-IW3_20231109T020004Z_20231114T103429Z_S1A_30_v1.0',
    'OPERA_L2_RTC-S1_T137-292339-IW3_20231121T020004Z_20231206T001313Z_S1A_30_v1.0',
    'OPERA_L2_RTC-S1_T137-292339-IW3_20231203T020004Z_20231203T100451Z_S1A_30_v1.0',
    'OPERA_L2_RTC-S1_T137-292339-IW3_20231203T020004Z_20231203T122715Z_S1A_30_v1.0',
    'OPERA_L2_RTC-S1_T137-292339-IW3_20231215T020003Z_20231215T142550Z_S1A_30_v1.0',
    'OPERA_L2_RTC-S1_T137-292339-IW3_20231227T020002Z_20231230T152233Z_S1A_30_v1.0',
    'OPERA_L2_RTC-S1_T137-292339-IW3_20240108T020002Z_20240109T091409Z_S1A_30_v1.0',
    'OPERA_L2_RTC-S1_T137-292339-IW3_20240120T020002Z_20240120T142521Z_S1A_30_v1.0',
    'OPERA_L2_RTC-S1_T137-292339-IW3_20240201T020001Z_20240201T081115Z_S1A_30_v1.0',
    'OPERA_L2_RTC-S1_T137-292339-IW3_20240201T020001Z_20240201T114538Z_S1A_30_v1.0'
])
opera_rtc_ids

['OPERA_L2_RTC-S1_T137-292339-IW3_20231004T020005Z_20240122T203351Z_S1A_30_v1.0',
 'OPERA_L2_RTC-S1_T137-292339-IW3_20231016T020005Z_20231016T154509Z_S1A_30_v1.0',
 'OPERA_L2_RTC-S1_T137-292339-IW3_20231028T020005Z_20231029T045555Z_S1A_30_v1.0',
 'OPERA_L2_RTC-S1_T137-292339-IW3_20231109T020004Z_20231114T103429Z_S1A_30_v1.0',
 'OPERA_L2_RTC-S1_T137-292339-IW3_20231121T020004Z_20231206T001313Z_S1A_30_v1.0',
 'OPERA_L2_RTC-S1_T137-292339-IW3_20231203T020004Z_20231203T100451Z_S1A_30_v1.0',
 'OPERA_L2_RTC-S1_T137-292339-IW3_20231203T020004Z_20231203T122715Z_S1A_30_v1.0',
 'OPERA_L2_RTC-S1_T137-292339-IW3_20231215T020003Z_20231215T142550Z_S1A_30_v1.0',
 'OPERA_L2_RTC-S1_T137-292339-IW3_20231227T020002Z_20231230T152233Z_S1A_30_v1.0',
 'OPERA_L2_RTC-S1_T137-292339-IW3_20240108T020002Z_20240109T091409Z_S1A_30_v1.0',
 'OPERA_L2_RTC-S1_T137-292339-IW3_20240120T020002Z_20240120T142521Z_S1A_30_v1.0',
 'OPERA_L2_RTC-S1_T137-292339-IW3_20240201T020001Z_20240201T081115Z_S1A_30_v1.0',
 'OPERA_L2_RTC-S

**Create a pandas DataFrame of the time series**

In [13]:
def get_dt(opera_id, date_regex):
    acquisition_time = re.search(date_regex, opera_id)
    try:
        return acquisition_time.group(0)
    except AttributeError:
        raise Exception(f"Acquisition timestamp not found in scene ID: {opera_id}") 

acquisition_date_regex = r"(?<=_)\d{8}T\d{6}Z(?=_\d{8}T\d{6})"
process_dt_regex = r"(?<=\d{8}T\d{6}Z_)\d{8}T\d{6}Z(?=_S1)"

acquisition_dt = pd.to_datetime([get_dt(id, acquisition_date_regex) for id in opera_rtc_ids])
process_dt = pd.to_datetime([get_dt(id, process_dt_regex) for id in opera_rtc_ids])

times_series_df = (pd.DataFrame(data={
    'OPERA L2-RTC-S1 ID': opera_rtc_ids, 
    'AcquisitionDateTime': acquisition_dt,
    'ProcessDateTime': process_dt
})
.sort_values(by='ProcessDateTime')
.drop_duplicates(subset=['AcquisitionDateTime'], keep='last')
.drop('ProcessDateTime', axis=1)
.sort_values(by='AcquisitionDateTime')
.reset_index(drop=True))

times_series_df

,OPERA L2-RTC-S1 ID,AcquisitionDateTime
0,OPERA_L2_RTC-S1_T137-292339-IW3_20231004T02000...,2023-10-04 02:00:05+00:00
1,OPERA_L2_RTC-S1_T137-292339-IW3_20231016T02000...,2023-10-16 02:00:05+00:00
2,OPERA_L2_RTC-S1_T137-292339-IW3_20231028T02000...,2023-10-28 02:00:05+00:00
3,OPERA_L2_RTC-S1_T137-292339-IW3_20231109T02000...,2023-11-09 02:00:04+00:00
4,OPERA_L2_RTC-S1_T137-292339-IW3_20231121T02000...,2023-11-21 02:00:04+00:00
5,OPERA_L2_RTC-S1_T137-292339-IW3_20231203T02000...,2023-12-03 02:00:04+00:00
6,OPERA_L2_RTC-S1_T137-292339-IW3_20231215T02000...,2023-12-15 02:00:03+00:00
7,OPERA_L2_RTC-S1_T137-292339-IW3_20231227T02000...,2023-12-27 02:00:02+00:00
8,OPERA_L2_RTC-S1_T137-292339-IW3_20240108T02000...,2024-01-08 02:00:02+00:00
9,OPERA_L2_RTC-S1_T137-292339-IW3_20240120T02000...,2024-01-20 02:00:02+00:00


**Build a time series of both polarizations in an xarray Dataset**

In [15]:
fs = s3fs.S3FileSystem(key=creds['accessKeyId'], secret=creds['secretAccessKey'], token=creds['sessionToken'])

polarizations = ['VV', 'VH']
da_stack = []

for t, row in times_series_df.iterrows():
    opera_id = row['OPERA L2-RTC-S1 ID']
    time = pd.to_datetime(row['AcquisitionDateTime'])
    polarization_stack = []

    for polarization in polarizations:
        filename = f"{opera_id}_{polarization}.tif"
        object_key = f"OPERA_L2_RTC-S1/{opera_id}/{filename}"
        s3_path = f"s3://{event['Bucket']}/{object_key}"

        with fs.open(s3_path, mode='rb') as f:
            da = xr.open_dataarray(f, engine="rasterio")
            da = da.expand_dims(time=pd.Index([time], name='time'))
            polarization_stack.append(da)

    da_polarized = xr.concat(polarization_stack, dim=pd.Index(polarizations, name='polarization'))
    da_stack.append(da_polarized)

ds = xr.concat(da_stack, dim='time')
ds

<xarray.DataArray 'band_data' (polarization: 2, time: 11, band: 1, y: 1609,
                               x: 3106)>
array([[[[[nan, nan, nan, ..., nan, nan, nan],
          [nan, nan, nan, ..., nan, nan, nan],
          [nan, nan, nan, ..., nan, nan, nan],
          ...,
          [nan, nan, nan, ..., nan, nan, nan],
          [nan, nan, nan, ..., nan, nan, nan],
          [nan, nan, nan, ..., nan, nan, nan]]],


        [[[nan, nan, nan, ..., nan, nan, nan],
          [nan, nan, nan, ..., nan, nan, nan],
          [nan, nan, nan, ..., nan, nan, nan],
          ...,
          [nan, nan, nan, ..., nan, nan, nan],
          [nan, nan, nan, ..., nan, nan, nan],
          [nan, nan, nan, ..., nan, nan, nan]]],


        [[[nan, nan, nan, ..., nan, nan, nan],
          [nan, nan, nan, ..., nan, nan, nan],
...
          [nan, nan, nan, ..., nan, nan, nan],
          [nan, nan, nan, ..., nan, nan, nan]]],


        [[[nan, nan, nan, ..., nan, nan, nan],
          [nan, nan, nan, ..., nan, nan, nan],
          [nan, nan, nan, ..., nan, nan, nan],
          ...,
          [nan, nan, nan, ..., nan, nan, nan],
          [nan, nan, nan, ..., nan, nan, nan],
          [nan, nan, nan, ..., nan, nan, nan]]],


        [[[nan, nan, nan, ..., nan, nan, nan],
          [nan, nan, nan, ..., nan, nan, nan],
          [nan, nan, nan, ..., nan, nan, nan],
          ...,
          [nan, nan, nan, ..., nan, nan, nan],
          [nan, nan, nan, ..., nan, nan, nan],
          [nan, nan, nan, ..., nan, nan, nan]]]]], dtype=float32)
Coordinates:
  * time          (time) object 1696384805000000000 ... 1706752801000000000
  * band          (band) int64 1
  * x             (x) float64 7.713e+05 7.713e+05 ... 8.644e+05 8.644e+05
  * y             (y) float64 4.228e+06 4.228e+06 ... 4.18e+06 4.18e+06
    spatial_ref   int64 0
  * polarization  (polarization) object 'VV' 'VH'
Attributes: (12/98)
    ABSOLUTE_ORBIT_NUMBER:                                                   ...
    ACQUISITION_MODE:                                                        ...
    AREA_OR_POINT:                                                           ...
    BOUNDING_BOX:                                                            ...
    BOUNDING_BOX_EPSG_CODE:                                                  ...
    BOUNDING_BOX_PIXEL_COORDINATE_CONVENTION:                                ...
    ...                                                                               ...
    STATIC_LAYERS_DATA_ACCESS:                                               ...
    SUB_SWATH_ID:                                                            ...
    TRACK_NUMBER:                                                            ...
    ZERO_DOPPLER_END_TIME:                                                   ...
    ZERO_DOPPLER_START_TIME:                                                 ...
    long_name:                                                               ...

**Access the VV time series**

In [16]:
my_time = pd.to_datetime('20231203T020004Z')
ds.sel(polarization='VV')

<xarray.DataArray 'band_data' (time: 11, band: 1, y: 1609, x: 3106)>
array([[[[nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         ...,
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan]]],


       [[[nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         ...,
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan]]],


       [[[nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
...
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan]]],


       [[[nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         ...,
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan]]],


       [[[nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         ...,
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan]]]], dtype=float32)
Coordinates:
  * time          (time) object 1696384805000000000 ... 1706752801000000000
  * band          (band) int64 1
  * x             (x) float64 7.713e+05 7.713e+05 ... 8.644e+05 8.644e+05
  * y             (y) float64 4.228e+06 4.228e+06 ... 4.18e+06 4.18e+06
    spatial_ref   int64 0
    polarization  <U2 'VV'
Attributes: (12/98)
    ABSOLUTE_ORBIT_NUMBER:                                                   ...
    ACQUISITION_MODE:                                                        ...
    AREA_OR_POINT:                                                           ...
    BOUNDING_BOX:                                                            ...
    BOUNDING_BOX_EPSG_CODE:                                                  ...
    BOUNDING_BOX_PIXEL_COORDINATE_CONVENTION:                                ...
    ...                                                                               ...
    STATIC_LAYERS_DATA_ACCESS:                                               ...
    SUB_SWATH_ID:                                                            ...
    TRACK_NUMBER:                                                            ...
    ZERO_DOPPLER_END_TIME:                                                   ...
    ZERO_DOPPLER_START_TIME:                                                 ...
    long_name:                                                               ...

**Access the VH RTC from the 1st timestep**

In [17]:
ds.sel(polarization='VH').isel(time=0)

<xarray.DataArray 'band_data' (band: 1, y: 1609, x: 3106)>
array([[[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]]], dtype=float32)
Coordinates:
    time          datetime64[ns] 2023-10-04T02:00:05
  * band          (band) int64 1
  * x             (x) float64 7.713e+05 7.713e+05 ... 8.644e+05 8.644e+05
  * y             (y) float64 4.228e+06 4.228e+06 ... 4.18e+06 4.18e+06
    spatial_ref   int64 0
    polarization  <U2 'VH'
Attributes: (12/98)
    ABSOLUTE_ORBIT_NUMBER:                                                   ...
    ACQUISITION_MODE:                                                        ...
    AREA_OR_POINT:                                                           ...
    BOUNDING_BOX:                                                            ...
    BOUNDING_BOX_EPSG_CODE:                                                  ...
    BOUNDING_BOX_PIXEL_COORDINATE_CONVENTION:                                ...
    ...                                                                               ...
    STATIC_LAYERS_DATA_ACCESS:                                               ...
    SUB_SWATH_ID:                                                            ...
    TRACK_NUMBER:                                                            ...
    ZERO_DOPPLER_END_TIME:                                                   ...
    ZERO_DOPPLER_START_TIME:                                                 ...
    long_name:                                                               ...